This notebook identifies a list of all single agent ICS inhalers that have not been categorised as high dose i.e. low and Medium dose inhalers. This is a pragmatic approach to work out a steroid load for a patient without using dose syntax.

- [All ICS inhalers](#ai)
- [low and medium dose ICS inhalers](#lm)

In [1]:
#import libraries
from ebmdatalab import bq
import os
import pandas as pd

## All ICS Single Agent Inhalers <a id='ai'></a>

In [12]:


sql = '''
WITH bnf_codes AS (  
  SELECT DISTINCT bnf_code FROM measures.dmd_objs_with_form_route WHERE 
  (bnf_code LIKE '0302000C0%' OR #BNF Beclometasone dipropionate
  bnf_code LIKE '0301011AB%'  OR #BNF BeclometDiprop/Formoterol/Glycopyrronium",
  bnf_code LIKE '0302000K0%'  OR #BNF budesonide
  bnf_code LIKE '0302000U0%'  OR #BNF Ciclesonide
  bnf_code LIKE '0302000V0%'  OR #BNF Fluticasone furoate 
  bnf_code LIKE '0302000N0%'  OR #BNF Fluticasone propionate 
  bnf_code LIKE '0302000R0%')   #BNF Mometasone Furoate
  AND
  (form_route LIKE '%pressurizedinhalation.inhalation' OR form_route LIKE 'powderinhalation.inhalation%')
   )

SELECT "vmp" AS type, vmp.id as id, vmp.bnf_code as bnf_code, vmp.nm as nm
FROM dmd.vmp as vmp
INNER JOIN dmd.vpi as vpi
ON
vmp.id = vpi.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)
GROUP BY vmp.id, vmp.bnf_code, vmp.nm
HAVING COUNT(ing) = 1

UNION ALL

SELECT "amp" AS type, amp.id as id, amp.bnf_code as bnf_code, amp.descr as nm
FROM dmd.amp as amp
INNER JOIN dmd.vpi as vpi
ON
amp.vmp = vpi.vmp
WHERE bnf_code IN (SELECT * FROM bnf_codes)
GROUP BY amp.id, amp.bnf_code, amp.descr
HAVING COUNT(ing) = 1

ORDER BY type, bnf_code, id'''

all_inhaler_ics_single_agent = bq.cached_read(sql, csv_path=os.path.join('..','data','all_inhaler_ics_single_agent.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
all_inhaler_ics_single_agent.info()

Downloading: 100%|██████████| 147/147 [00:00<00:00, 1058.51rows/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      147 non-null    object
 1   id        147 non-null    int64 
 2   bnf_code  147 non-null    object
 3   nm        147 non-null    object
dtypes: int64(1), object(3)
memory usage: 4.7+ KB


In [13]:
#import csv from other notebook dealing with high dose
dose_high_ics_single_agent = pd.read_csv('../data/highdose_inhaledsteroid_singleagent_codelist.csv')
dose_high_ics_single_agent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      28 non-null     object
 1   id        28 non-null     int64 
 2   bnf_code  28 non-null     object
 3   nm        28 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.0+ KB


In [14]:
## here we merge and create an indicator to see which ones are in both
combine = pd.merge(all_inhaler_ics_single_agent,dose_high_ics_single_agent, how='outer', indicator=True)
combine

,type,id,bnf_code,nm,_merge
0,amp,3175711000001102,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (A A H Pharmaceuticals Ltd),left_only
1,amp,3175911000001100,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (Mylan),left_only
2,amp,3176711000001105,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (Kent Pharmaceuticals Ltd),left_only
3,amp,3177111000001107,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (Alliance Healthcare (Distribution) Ltd),left_only
4,amp,4773611000001100,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (Teva UK Ltd),left_only
5,amp,11400011000001108,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (Almus Pharmaceuticals Ltd),left_only
6,amp,17793411000001101,0302000C0AAAAAA,Beclometasone 50micrograms/dose inhaler (Phoenix Healthcare Distribution Ltd),left_only
7,amp,3178211000001102,0302000C0AAABAB,Beclometasone 100micrograms/dose inhaler (A A H Pharmaceuticals Ltd),left_only
8,amp,3178411000001103,0302000C0AAABAB,Beclometasone 100micrograms/dose inhaler (Mylan),left_only
9,amp,3179011000001102,0302000C0AAABAB,Beclometasone 100micrograms/dose inhaler (Alliance Healthcare (Distribution) Ltd),left_only


## Low and medium dose Single Dose ICS <a id='lm'></a>

In [15]:
#ones that = left only are low medium dose
low_med_ics_single_agent = combine.loc[(combine['_merge'] == "left_only")]
low_med_ics_single_agent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 0 to 146
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   type      119 non-null    object  
 1   id        119 non-null    int64   
 2   bnf_code  119 non-null    object  
 3   nm        119 non-null    object  
 4   _merge    119 non-null    category
dtypes: category(1), int64(1), object(3)
memory usage: 4.9+ KB


In [16]:
low_med_ics_single_agent.sort_values(["type", "nm"], inplace=True)
low_med_ics_single_agent.drop('_merge', 1, inplace=True)

<ipython-input-16-46863b4a7a70>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_med_ics_single_agent.sort_values(["type", "nm"], inplace=True)
/root/.pyenv/versions/3.8.1/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [17]:
low_med_ics_single_agent

,type,id,bnf_code,nm
31,amp,3181411000001102,0302000C0BEACAT,AeroBec 100 Autohaler (Meda Pharmaceuticals Ltd)
94,amp,9004211000001109,0302000U0BBABAB,Alvesco 160 inhaler (AstraZeneca UK Ltd)
93,amp,9003911000001102,0302000U0BBAAAA,Alvesco 80 inhaler (AstraZeneca UK Ltd)
48,amp,3112511000001109,0302000C0BIAEBJ,Asmabec 100 Clickhaler (Focus Pharmaceuticals Ltd)
47,amp,3111911000001108,0302000C0BIADBI,Asmabec 50 Clickhaler (Focus Pharmaceuticals Ltd)
91,amp,4045711000001107,0302000R0BBAAAA,Asmanex 200micrograms/dose Twisthaler (Merck Sharp & Dohme Ltd)
92,amp,4043811000001103,0302000R0BBACAC,Asmanex 400micrograms/dose Twisthaler (Merck Sharp & Dohme Ltd)
36,amp,3181711000001108,0302000C0BFAEAT,Beclazone 100 Easi-Breathe inhaler (Teva UK Ltd)
33,amp,3178811000001101,0302000C0BFABAB,Beclazone 100 inhaler (Teva UK Ltd)
38,amp,3179511000001105,0302000C0BFAGAR,Beclazone 200 inhaler (Teva UK Ltd)


In [20]:
low_med_ics_single_agent.to_csv(os.path.join('..','data','low_med_inhaledsteroid_singleagent_codelist.csv'))